<a href="https://colab.research.google.com/github/navrgithub/NLP_Authorship_Attribution/blob/main/RNN_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP group project/data')

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [ ]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',tokenizer_language='en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('final_task2_data.csv')

In [ ]:
df.head()

,Unnamed: 0,text,label
0,0,CMV: Fat Acceptance Tik Tokers and influencers...,0
1,1,CMV: It makes sense to compare apples and oranges,0
2,2,CMV: Dismissing an individual's experience bec...,0
3,3,"CMV: If pro-anorexia groups, suicide promotion...",0
4,4,CMV: Asian Americans generally prospering in t...,0


In [ ]:
df[['text']] = df[['text']]. astype(str)

In [ ]:
df = df[['text','label']]

In [ ]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm',  include_lengths = True)
LABEL = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)

pt = [TEXT.preprocess(x) for x in df['text']]

TEXT.build_vocab(
    preprocessed_text, 
    max_size = 30000,
    vectors='glove.6B.50d',
    unk_init = torch.Tensor.normal_)

In [ ]:
label_to_index= {label: index for index, label in enumerate(df['label'].unique())}

In [ ]:
df['label_new'] = df['label'].apply(lambda y: label_to_index[y])

class DataFrameDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                data.Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], fields)

In [ ]:
df_new=df[['text','label_new']]


In [ ]:
examples=[]
for i,row in df_new.iterrows():
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))
     

In [ ]:
itrr=0
for i in examples:
  if type(i.label) != int:
    itrr+=1
     

In [ ]:
dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])

In [ ]:
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.65, 0.15, 0.2], stratified=True, strata_field='label')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)
# device = 'cpu'
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cuda


In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        # 1. Initialize Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        # 2. Initialize RNN layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=n_layers, 
                          bidirectional=bidirectional, dropout=dropout)
        
        # 3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        # 4. Initialize Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        # text = [sent_len, batch_size]
        
        # 1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        embedded = self.dropout(self.embedding(text))
        
        # 2. Run the RNN along the sentences of length sent_len. 
        # output = [sent len, batch size, hid dim * num directions]; 
        # hidden = [num layers * num directions, batch size, hid dim]
        tl=text_lengths.to('cpu')
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, tl)
        packed_output, hidden = self.rnn(packed_embedded)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output)
        
        # 3. Get last forward (hidden[-1,:,:]) hidden layer and apply dropout
        last_forward_hidden = hidden[-1, :, :]
        last_forward_hidden = self.dropout(last_forward_hidden)
        
        return self.fc(last_forward_hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 256
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,713,359 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([30002, 50])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.7372,  0.3910,  0.5158],
        [ 0.1825, -0.0737,  0.3147,  ...,  1.3226, -2.6796, -0.1297],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.3260,  2.3001,  0.0365,  ..., -1.4911,  0.6114,  0.9878],
        [ 0.0854, -0.6890,  0.6959,  ..., -0.3637,  0.2291,  0.6251],
        [ 0.7506,  0.1679, -0.3358,  ...,  1.3170, -0.5806, -1.2388]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.3260,  2.3001,  0.0365,  ..., -1.4911,  0.6114,  0.9878],
        [ 0.0854, -0.6890,  0.6959,  ..., -0.3637,  0.2291,  0.6251],
        [ 0.7506,  0.1679, -0.3358,  ...,  1.3170, -0.5806, -1.2388]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss()

# device='cpu'
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # print(batch.text)
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        target = batch.label
        target = target.type(torch.LongTensor)

        # print(target)

        # print(predictions)
        
        loss = criterion(predictions.to(device), target.to(device))
        
        acc = multiclass_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()



    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 50
path = F"rnn_model_task2.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.511 | Train Acc: 85.06%
	 Val. Loss: 0.415 |  Val. Acc: 86.64%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.447 | Train Acc: 86.47%
	 Val. Loss: 0.416 |  Val. Acc: 86.58%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 0.418 | Train Acc: 86.63%
	 Val. Loss: 0.401 |  Val. Acc: 86.58%
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.417 | Train Acc: 86.72%
	 Val. Loss: 0.396 |  Val. Acc: 86.58%
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 0.414 | Train Acc: 86.72%
	 Val. Loss: 0.398 |  Val. Acc: 86.58%
Epoch: 06 | Epoch Time: 0m 11s
	Train Loss: 0.422 | Train Acc: 86.70%
	 Val. Loss: 0.396 |  Val. Acc: 86.58%
Epoch: 07 | Epoch Time: 0m 10s
	Train Loss: 0.408 | Train Acc: 86.64%
	 Val. Loss: 0.407 |  Val. Acc: 86.58%
Epoch: 08 | Epoch Time: 0m 9s
	Train Loss: 0.413 | Train Acc: 86.70%
	 Val. Loss: 0.397 |  Val. Acc: 86.58%
Epoch: 09 | Epoch Time: 0m 10s
	Train Loss: 0.418 | Train Acc: 86.69%
	 Val. Loss: 0.406 |  Val. Acc: 86.58%
Epoch: 10 | Epoch Ti

In [ ]:
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.457 | Test Acc: 86.18%
